In [4]:
import csv
input_path = "data/b0534_094_118_121_134_144_148_151_166.out"
def txt_to_csv(input_txt_path=input_path, output_csv_path="pure_proton.csv"):
    with open(input_txt_path, 'r') as infile, open(output_csv_path, 'w', newline='') as outfile:
        writer = csv.writer(outfile)

        # Define headers for CSV
        header = [
            "centroid_x", "centroid_y", "2a", "2b", "eccentricity",
            "radial_distance", "radial_angle", "orientation_angle", "alpha",
            "orientation_slope", "radial_slope", "total_active_pixels", "total_PE_count",
            "azimuthal_spread", "miss_distance", "edge_leakage_ratio", "brightest_pixel_distance",
            "brightest_pixel_value", "flag", "distribution_asymmetry",
            "frac2", "bt", "pt"
        ]
        writer.writerow(header)

        lines = infile.readlines()
        i = 0
        while i < len(lines) - 2:
            # Skip metadata line
            energy1 = lines[i + 1].strip().split()
            energy2 = lines[i + 2].strip().split()

            if len(energy1) == 23:
                writer.writerow(energy1)

            if len(energy2) == 23:
                writer.writerow(energy2)

            i += 3  # Move to next event (3 lines per event)

    print(f"✅ Conversion completed. CSV saved to: {output_csv_path}")
#txt_to_csv()  # Call the function to convert the file

In [15]:
import pandas as pd
from tqdm import tqdm

def extract_valid_events(csv_path="pure_proton.csv", output_csv="filtered_proton.csv"):
    # Load the CSV
    df = pd.read_csv(csv_path)

    # Initialize list to store valid events
    extracted_rows = []

    # Iterate through events (2 rows per event) with tqdm progress bar
    for i in tqdm(range(0, len(df), 2), desc="Processing events"):
        event_rows = df.iloc[i:i+2]

        # Skip if both rows have total_active_pixels == 0
        if (event_rows["total_active_pixels"] <= 6).all():
            continue

        # Select the row with flag == 1
        valid_row = event_rows[(event_rows["flag"] == 1) & (event_rows["total_active_pixels"] > 6)]

        if not valid_row.empty:
            row = valid_row.iloc[0]
            extracted_rows.append({
                "miss_distance": row["miss_distance"],
                "alpha": row["alpha"],
                "radial_distance": row["radial_distance"],
                "azimuthal_spread": row["azimuthal_spread"],
                "particle_type": 1
            })

    # Convert to final DataFrame
    final_df = pd.DataFrame(extracted_rows)

    # Save to CSV
    final_df.to_csv(output_csv, index=False)
    print(f"\n✅ Extracted {len(final_df)} valid events to '{output_csv}'")

# Call the function
extract_valid_events()


Processing events: 100%|██████████| 551028/551028 [00:58<00:00, 9429.15it/s] 



✅ Extracted 179795 valid events to 'filtered_proton.csv'


In [9]:
txt_to_csv(input_txt_path="data/s0534+2201_060_NOR_08112024_1_EVBdata_hillas_par.out", output_csv_path="source_1.csv")
extract_valid_events(csv_path="source_1.csv", output_csv="filtered_source_1.csv")

✅ Conversion completed. CSV saved to: source_1.csv


Processing events: 100%|██████████| 35000/35000 [00:03<00:00, 10191.30it/s]



✅ Extracted 10457 valid events to 'filtered_source_1.csv'


🔍 Showing 10 valid events with total_active_pixels > 6:



Processing events:   0%|          | 2056/551028 [00:00<00:52, 10444.03it/s]

 miss_distance  alpha  radial_distance  azimuthal_spread  total_active_pixels
          6.05  82.53             6.11              1.64                   12
------------------------------------------------------------
 miss_distance  alpha  radial_distance  azimuthal_spread  total_active_pixels
          7.17  85.49             7.19              1.23                    9
------------------------------------------------------------
 miss_distance  alpha  radial_distance  azimuthal_spread  total_active_pixels
          6.34  55.53             7.69              0.73                    8
------------------------------------------------------------
 miss_distance  alpha  radial_distance  azimuthal_spread  total_active_pixels
          0.23   1.73             7.55              0.42                    8
------------------------------------------------------------
 miss_distance  alpha  radial_distance  azimuthal_spread  total_active_pixels
          2.58  34.79             4.53              1.

Processing events: 100%|██████████| 551028/551028 [00:57<00:00, 9503.71it/s] 



✅ Extracted 179795 valid events to 'filtered_proton.csv'
